# Colab flow

### GPU usage

To use GPU in colab: `Edit > Notebook settings or Runtime>Change runtime type and select GPU as Hardware accelerator`.

### Setting repo

In [1]:
colab = False

For `colab=True` you need to rewrite the `branch` and `github_token` below.

In [2]:
if colab:
    !pip install condacolab # Handling conda environment installation
    import condacolab
    condacolab.install()
    !git clone -b branch https://github_token@github.com/Nixtla/nixtlats.git
    %cd nixtlats
    !mamba env update -n base -f environment.yml
    !pip install -e .

# Standard flow

In [18]:
import numpy as np
import pytorch_lightning as pl
import torch as t
from pytorch_lightning import loggers as pl_loggers
from pytorch_lightning import seed_everything

from nixtla.data.datasets.m4 import M4, M4Info, M4Evaluation
from nixtla.data.tsdataset import TimeSeriesDataset
from nixtla.data.tsloader import TimeSeriesLoader
from nixtla.models.esrnn.esrnn import ESRNN

In [3]:
seed_everything(117982, workers=True)

Global seed set to 117982


117982

## Import Yearly data

In [4]:
group = M4Info['Yearly']
Y_df, _, S_df = M4.load(directory='data', group=group.name)

In [5]:
train_ts_dataset = TimeSeriesDataset(Y_df=Y_df, S_df=S_df,
                                     ds_in_test=group.horizon,
                                     mode='full',
                                     window_sampling_limit=20_000, # To limit backprop time
                                     input_size=4,
                                     output_size=group.horizon,
                                     idx_to_sample_freq=1,
                                     len_sample_chunks=group.horizon * 3,
                                     complete_inputs=True, 
                                     skip_nonsamplable=True)

In [6]:
test_ts_dataset = TimeSeriesDataset(Y_df=Y_df, S_df=S_df,
                                    ds_in_test=0,
                                    mode='full',
                                    window_sampling_limit=500_000, # To limit backprop time
                                    input_size=4,
                                    output_size=group.horizon,
                                    idx_to_sample_freq=1,
                                    complete_inputs=True,
                                    complete_outputs=True,
                                    len_sample_chunks=group.horizon * 3,
                                    skip_nonsamplable=False,
                                    last_samplable_window=True)

In [7]:
train_ts_loader = TimeSeriesLoader(dataset=train_ts_dataset,
                                   batch_size=1024,
                                   eq_batch_size=True,
                                   num_workers=8,
                                   shuffle=False)

In [8]:
test_ts_loader = TimeSeriesLoader(dataset=test_ts_dataset,
                                  batch_size=1024,
                                  eq_batch_size=False,
                                  num_workers=8,
                                  shuffle=False)

In [9]:
model = ESRNN(n_series=group.n_ts,
              n_x=0, n_s=1,
              idx_to_sample_freq=1,
              input_size=4,
              output_size=group.horizon,
              learning_rate=1e-3,
              lr_scheduler_step_size=2,
              lr_decay=0.1,
              per_series_lr_multip=0.8,
              gradient_clipping_threshold=20,
              rnn_weight_decay=0,
              level_variability_penalty=0,
              testing_percentile=50,
              training_percentile=45,
              cell_type='LSTM',
              state_hsize=30,
              dilations=[[1, 2], [2, 6]],
              add_nl_layer=True,
              loss='SMYL',
              val_loss='SMAPE',
              seasonality=[1])

In [11]:
trainer = pl.Trainer(max_epochs=15, gpus=int(colab),
                     progress_bar_refresh_rate=10, deterministic=True)
trainer.fit(model, train_ts_loader)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type   | Params
---------------------------------
0 | esrnn | _ESRNN | 96.9 K
---------------------------------
96.9 K    Trainable params
0         Non-trainable params
96.9 K    Total params
0.388     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Global seed set to 117982


Training: 0it [00:00, ?it/s]

In [12]:
outputs = trainer.predict(model, test_ts_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: 23it [00:00, ?it/s]

In [13]:
_, y_hat = zip(*outputs)

In [14]:
y_hat = t.cat(y_hat)[:, -1].cpu().numpy()

In [15]:
y = Y_df.groupby('unique_id').tail(group.horizon)['y'].values.reshape(-1, group.horizon)

In [19]:
M4Evaluation.evaluate('data', 'Yearly', y_hat)

,SMAPE,MASE,OWA
Yearly,12.115387,2.923945,0.73853
